<a href="https://colab.research.google.com/github/maicon-reis/outspoken-market-na-pratica/blob/main/25_Ajuste_Polinomial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q yfinance

In [ ]:
# Carregando as bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, r2_score
from sympy import S, symbols, printing
import sklearn.metrics as metrics
import yfinance as yf
import warnings
import datetime as dt
warnings.filterwarnings("ignore")

In [ ]:
# Função que cria a variável IFR
def criaIFR(data, p_rsi = 14):
    #
    # Essa função cria uma nova variável, IFR, e retorna o dataframe recebido
    # acrescido da variável criada.
    # Parâmetros de entrada:
    #   data -> Dataframe contendo as colunas de origem e as variáveis criadas.
    #   p_rsi -> Inteiro correspondendo ao período utilizado para criar a  
    #            variável.
    # Return:
    #   A função retorna o dataframe data acrescido da coluna ifr.
    #
    data['delta'] = data['Adj Close'].diff()
    data.dropna(inplace=True)
    data.loc[:, 'positivos'] = np.where(data['delta'] < 0, 0, data['delta'])
    data.loc[:, 'negativos'] = np.where(data['delta'] > 0, 0, data['delta'])
    data['mediaPos'] = data['positivos'].rolling(window=p_rsi).mean()
    data['mediaNeg'] = abs(data['negativos'].rolling(window=p_rsi).mean())
    data.dropna(inplace=True)
    data['fr'] = data['mediaPos'] / data['mediaNeg']
    data['ifr'] = 100.00 - (100.0 / (1.0 + data['fr']))
    data.drop(['delta', 'positivos', 'negativos', 'mediaPos', 'mediaNeg', 'fr'], axis=1,
              inplace=True)

    return data

In [ ]:
# Importando a base de dados da biblioteca Yfinance
ativo = "VALE3.SA"
inicio = "2000-01-02"
fim = dt.datetime.today()
df = yf.download(ativo, data_source = "yahoo", start = inicio, end = fim)
df.head()

In [ ]:
# Criando coluna  de Retorno e Alvo
p = 5
p_alvo = 0.0025 #0.25%

# Construção dos alvos
# Alvo - Retorno
df['Retorno'] = df['Adj Close'].pct_change(p)
df['Alvo'] = df['Retorno'].shift(-p)
df['Alvo_cat'] = np.where(df['Alvo'] > p_alvo, 1
                          , np.where(df['Alvo'] < -p_alvo, -1, 0))

In [ ]:
# Criando variáveis que serão utilizadas
p_mm = 21
df['mm'] = df['Adj Close'].rolling(p_mm).mean() # cria média móvel
df['dist_mm'] = round((df['Adj Close']/df['mm']-1), 3) # distância do preço para a média
df = criaIFR(df) # Calcula o ifr
df['ifr_mm'] = round(df['dist_mm']*df['ifr'], 3) # ifr ponderado da média móvel

# Visualizando os dados
df.head()

### **Ajuste de Polinômios**

O polinomio que será ajustado será o IFR pela distância da média, ou seja, o x será o IFR (```df["ifr"]```) e y será a distância da média móvel em termos percentuais (```df["dist_mm"] * 100```).

In [ ]:
x = df['ifr']
y = df['dist_mm'].values * 100

A utilização da função numpy ```polyfit``` realiza o ajusto polinomial minimizando os erros quadrados.

$$E=\sum_{j=0}^k|p(x_j)-y_j|^2$$

A função recebe, dentre os parâmetros, dois arrays ```x``` e ```y```, correspondendo às variáveis dependentes e independentes e deg, grau de polinomial que se deseja ajustar.

A utilização dessa função retorna um vetor de coeficientes do polinômio ```p``` que minimiza o erro quadrado na ordem indicada.

Já a função numpy ```poly1d``` é usda para construir a equação de ajuste polinomial.

Esta função recebe os coeficientes do do polinômio.

In [ ]:
grau = 3
ajuste = np.polyfit(x=x, y=y, deg=grau) # ajusta o polinômio para reduzir o erro
polinomio = np.poly1d(ajuste) # serve para construir o polinômio
print("{}".format(polinomio)) # Visualizando a equação de ajuste polinomial

### **Verificando o R2 para  a equação de ajuste polinomial**

In [ ]:
R2 = round(r2_score(y, polinomio(x)), 3)
print("R2 Ibov: {}".format(R2))

In [ ]:
# Verificando o resultado da equação para 3 valores aleatórios
print("Para IFR = 30, dist_mm = {}".format(round(polinomio(30), 3)))
print("Para IFR = 50, dist_mm = {}".format(round(polinomio(50), 3)))
print("Para IFR = 70, dist_mm = {}".format(round(polinomio(70), 3)))

### **Representando a Curva de Ajuste Polinomial**

In [ ]:
equacao = sum(S("{:.2E}".format(v))*symbols("x")**i for i, v in enumerate(ajuste[::-1]))
eq_latex = printing.latex(equacao)
print(eq_latex)

In [ ]:
# plot do polinomio
ref = np.linspace(min(x), max(x), len(x))

with plt.style.context("fivethirtyeight"):
    plt.figure(figsize=(10,8))
    plt.scatter(x, y, c="red", label="IFR14 x Distância da Média", s=9)
    plt.plot(ref, polinomio(ref), linestyle="-", lw=5, c="black")
    plt.xlabel("IFR 14", fontsize=15)
    plt.ylabel("Distância MM 21(%)", fontsize=15)
    plt.annotate("Equação: ""${}$".format(eq_latex), (10,40))
    plt.annotate("R2 =" + str(R2), (10, 37))
    plt.legend(fontsize=15)
    plt.title("Curva Polinominal - IFR 14 X Distância da Média \n Ibov")

### **Trabalhando com os Resíduos**

In [ ]:
# Criando uma coluna contendo os resíduos da equação
df['residuos'] = df['dist_mm'] * 100 - polinomio(df['ifr'])

Associando a análise dos resíduos com um alvo categórico de 5 períodos, percebemos os resíduos representam uma reversão à média. Quando os resíduos forem superiores à 

In [ ]:
pd.pivot_table(df, index=['Alvo_cat'], aggfunc = {"residuos": [np.median]})

In [ ]:
df['residuo_cat'] = np.where(df['residuos'].between(0.031428, 0.086636), -1
                             , np.where(df['residuos'] < -0.098292, 1, 0))

print(confusion_matrix(df['Alvo_cat'], df['residuo_cat']))
print()
print('------------------------------------------------------------------')
print(classification_report(df['Alvo_cat'], df['residuo_cat']))